In [ ]:
from pathlib import Path
import os
from itertools import combinations
import math

import numpy as np

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input24.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input24_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data

# Part 1

In [ ]:
hailstones = []
for line in data:
    pos, vel = line.split(" @ ")
    pos = [int(x) for x in pos.split(", ")]
    vel = [int(x) for x in vel.split(", ")]
    hailstone = [pos, vel]
    hailstones.append(hailstone)

In [ ]:
hailstones

In [ ]:
lower = 200000000000000
upper = 400000000000000

num_within_area = 0

# assume either x- or y-velocity is nonzero

for h1, h2 in combinations(hailstones, 2):
    (x1, y1, z1), (u1, v1, w1) = h1
    (x2, y2, z2), (u2, v2, w2) = h2

    """
    x_1 * t_1 * u1 = x_2 + t_2 * u_2
    y_1 + t_1 * v1 = y_2 + t_2 * v_2

    Write this as A * t = b.
    Then solve by matrix inversion of A.
    """

    det = -u1 * v2 + u2 * v1
    if det != 0:
        t1 = 1 / det * (-v2 * (x2 - x1) + u2 * (y2 - y1))
        t2 = 1 / det * (-v1 * (x2 - x1) + u1 * (y2 - y1))
        if t1 > 0 and t2 > 0:
            # in the future for both hailstones
            x_intersection = x1 + t1 * u1
            y_intersection = y1 + t1 * v1

            if lower <= x_intersection <= upper and lower <= y_intersection <= upper:
                num_within_area += 1
    else:
        if u2 != 0:
            if -v2 * (x1 - x2) == (y2 - y1) * u2:
                # lines are the same, infinitely many intersections (otherwise parallel but distinct lines, no intersections)
                num_within_area += 1
        else:
            if x1 == x2:
                # lines are the same, infinitely many intersections (otherwise parallel but distinct lines, no intersections)
                num_within_area += 1

In [ ]:
num_within_area

# Part 2

In [ ]:
hailstones = sorted(hailstones, key=lambda x: x[0][0])

In [ ]:
# Hailstone i has position (x_i, y_i, z_i) and velocity (u_i, v_i, z_i)
# Say hailstone to be thrown has position (a, b, c) and velocity (d, e, f)
# Set the first two coordinates equal and eliminate t. Then get an equation in a, b, d, e.
# Do the same for hailstone j.
# Subtract the resulting two equations and get a linear equation in a, b, d, e.
# Do the same for three other pairs of hailstones.
# Now solve the system of equations using matrix inversion.

total = 0

# Take average due to numerical issues
# This could be avoided using symbolic computation (Python package sympy)
for offset in range(len(hailstones) - 7):

    x = [h[0][0] for h in hailstones[offset:]]
    y = [h[0][1] for h in hailstones[offset:]]
    z = [h[0][2] for h in hailstones[offset:]]

    u = [h[1][0] for h in hailstones[offset:]]
    v = [h[1][1] for h in hailstones[offset:]]
    w = [h[1][2] for h in hailstones[offset:]]

    A = np.empty((4, 4))
    for i in range(4):
        A[i][0] = v[2 * i] - v[2 * i + 1]
        A[i][1] = u[2 * i + 1] - u[2 * i]
        A[i][2] = y[2 * i + 1] - y[2 * i]
        A[i][3] = x[2 * i] - x[2 * i + 1]

    b = np.empty((4, 1))
    for i in range(4):
        b[i] = x[2 * i] * v[2 * i] - y[2 * i] * u[2 * i] - x[2 * i + 1] * v[2 * i + 1] + y[2 * i + 1] * u[2 * i + 1]

    res = np.linalg.inv(A) @ b
    total += res

In [ ]:
res_avg = total / (len(hailstones) - 7)
res_avg

In [ ]:
d = -63
e = -301

In [ ]:
# Try out all candidates close to numerical average
a_prime = math.floor(res_avg[0, 0])
b_prime = math.floor(res_avg[1, 0])

corr_offsets = [-3, -2, -1, 0, 1, 2, 3]

corr_offset_results = {}

for corr_a in corr_offsets:
    for corr_b in corr_offsets:

        a_corr = a_prime + corr_a
        b_corr = b_prime + corr_b


        diff_total = 0
        for i in range(5):
            t = (a_corr - x[i]) / (u[i] - d)
            diff = (y[i] + t * v[i]) - (b_corr + t * e)
            diff_total += diff

        if diff_total == 0:
            print(a_corr, b_corr)
        corr_offset_results[(a_corr, b_corr)] = abs(diff_total)

In [ ]:
a = 309991770591665
b = 460585296453281

In [ ]:
t_0 = (a - x[0]) / (u[0] - d)
t_0

In [ ]:
t_1 = (a - x[1]) / (u[1] - d)
t_1

In [ ]:
f = (z[0] + t_0 * w[0] - z[1] - t_1 * w[1]) / (t_0 - t_1)
f

In [ ]:
c = z[0] + t_0 * w[0] - t_0 * f
c

In [ ]:
a + b + c